In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
df = pd.read_csv("output/ipl_ball_by_ball_output.csv")

In [3]:
df.shape

(260759, 18)

In [4]:
df.head()

,date,match_number,innings,over,batter,bowler,non_striker,runs.batter,runs.extras,runs.total,extras.legbyes,extras.wides,extras.byes,extras.noballs,wicket.kind,wicket.player_out,wicket.fielders,wickets
0,2008-04-18,1,1,0,SC Ganguly,P Kumar,BB McCullum,0,1,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2008-04-18,1,1,0,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2008-04-18,1,1,0,BB McCullum,P Kumar,SC Ganguly,0,1,1,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2008-04-18,1,1,0,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2008-04-18,1,1,0,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
summary_df = pd.read_csv("output/ipl_summary.csv")

In [6]:
summary_df["match_number"] = summary_df["match_number"].astype("str")

In [7]:
summary_df.shape

(1095, 20)

In [8]:
summary_df.head()

,date,match_number,city,team_1,team_2,outcome_winner,player_of_match,toss_winner,toss_decision,officials_match_referees,officials_reserve_umpires,officials_tv_umpires,officials_umpires_1,officials_umpires_2,outcome_by_wickets,season,venue,outcome_by_runs,event_stage,outcome_method
0,2008-04-18,1,Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,BB McCullum,Royal Challengers Bangalore,field,J Srinath,VN Kulkarni,AM Saheba,Asad Rauf,RE Koertzen,NaN,2008,MChinnaswamyStadium,140.0,group_stage,regular
1,2008-04-19,2,Chandigarh,Chennai Super Kings,Kings XI Punjab,Chennai Super Kings,MEK Hussey,Chennai Super Kings,bat,S Venkataraghavan,MSS Ranawat,RB Tiffin,MR Benson,SL Shastri,NaN,2008,PunjabCricketAssociationStadium,33.0,group_stage,regular
2,2008-04-19,3,Delhi,Rajasthan Royals,Delhi Daredevils,Delhi Daredevils,MF Maharoof,Rajasthan Royals,bat,GR Viswanath,NaN,IL Howell,Aleem Dar,GA Pratapkumar,9.0,2008,FerozShahKotla,NaN,group_stage,regular
3,2008-04-20,4,Kolkata,Deccan Chargers,Kolkata Knight Riders,Kolkata Knight Riders,DJ Hussey,Deccan Chargers,bat,FM Engineer,F Gomes,Asad Rauf,BF Bowden,K Hariharan,5.0,2008,EdenGardens,NaN,group_stage,regular
4,2008-04-20,5,Mumbai,Mumbai Indians,Royal Challengers Bangalore,Royal Challengers Bangalore,MV Boucher,Mumbai Indians,bat,J Srinath,SN Bandekar,AV Jayaprakash,SJ Davis,DJ Harper,5.0,2008,WankhedeStadium,NaN,group_stage,regular


In [9]:
def get_items_by_team(field, agg="sum"):
    if agg == "sum":
        grouped_df = df.groupby(["date", "match_number", "innings"])[field].sum()
    elif agg == "max":
        grouped_df = df.groupby(["date", "match_number", "innings"])[field].max()

    grouped_df = grouped_df.reset_index()

    df_pivot = grouped_df.pivot(
        index=["date", "match_number"], columns="innings", values=field
    )

    df_pivot.reset_index(inplace=True)

    df_pivot = df_pivot.rename(columns={1: "team_1_" + field, 2: "team_2_" + field})

    return df_pivot

In [10]:
df_pivot = get_items_by_team("runs.total")
df_merged = pd.merge(summary_df, df_pivot, how="outer", on=["date", "match_number"])

In [11]:
df_pivot = get_items_by_team("runs.extras")
df_merged = pd.merge(df_merged, df_pivot, how="outer", on=["date", "match_number"])

In [12]:
df_pivot = get_items_by_team("extras.legbyes")
df_merged = pd.merge(df_merged, df_pivot, how="outer", on=["date", "match_number"])

In [13]:
df_pivot = get_items_by_team("extras.wides")
df_merged = pd.merge(df_merged, df_pivot, how="outer", on=["date", "match_number"])

In [14]:
df_pivot = get_items_by_team("extras.byes")
df_merged = pd.merge(df_merged, df_pivot, how="outer", on=["date", "match_number"])

In [15]:
df_pivot = get_items_by_team("extras.noballs")
df_merged = pd.merge(df_merged, df_pivot, how="outer", on=["date", "match_number"])

In [16]:
df_pivot = get_items_by_team("over", "max")
df_merged = pd.merge(df_merged, df_pivot, how="outer", on=["date", "match_number"])

In [17]:
pd.set_option("display.max_columns", 35)
df_merged.head()

,date,match_number,city,team_1,team_2,outcome_winner,player_of_match,toss_winner,toss_decision,officials_match_referees,officials_reserve_umpires,officials_tv_umpires,officials_umpires_1,officials_umpires_2,outcome_by_wickets,season,venue,outcome_by_runs,event_stage,outcome_method,team_1_runs.total,team_2_runs.total,team_1_runs.extras,team_2_runs.extras,team_1_extras.legbyes,team_2_extras.legbyes,team_1_extras.wides,team_2_extras.wides,team_1_extras.byes,team_2_extras.byes,team_1_extras.noballs,team_2_extras.noballs,team_1_over,team_2_over
0,2008-04-18,1,Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,BB McCullum,Royal Challengers Bangalore,field,J Srinath,VN Kulkarni,AM Saheba,Asad Rauf,RE Koertzen,NaN,2008,MChinnaswamyStadium,140.0,group_stage,regular,222.0,82.0,17.0,19.0,4.0,8.0,9.0,11.0,4.0,0.0,0.0,0.0,19.0,15.0
1,2008-04-19,2,Chandigarh,Chennai Super Kings,Kings XI Punjab,Chennai Super Kings,MEK Hussey,Chennai Super Kings,bat,S Venkataraghavan,MSS Ranawat,RB Tiffin,MR Benson,SL Shastri,NaN,2008,PunjabCricketAssociationStadium,33.0,group_stage,regular,240.0,207.0,6.0,11.0,2.0,4.0,3.0,5.0,0.0,2.0,1.0,0.0,19.0,19.0
2,2008-04-19,3,Delhi,Rajasthan Royals,Delhi Daredevils,Delhi Daredevils,MF Maharoof,Rajasthan Royals,bat,GR Viswanath,NaN,IL Howell,Aleem Dar,GA Pratapkumar,9.0,2008,FerozShahKotla,NaN,group_stage,regular,129.0,132.0,7.0,10.0,3.0,0.0,3.0,10.0,1.0,0.0,0.0,0.0,19.0,15.0
3,2008-04-20,4,Kolkata,Deccan Chargers,Kolkata Knight Riders,Kolkata Knight Riders,DJ Hussey,Deccan Chargers,bat,FM Engineer,F Gomes,Asad Rauf,BF Bowden,K Hariharan,5.0,2008,EdenGardens,NaN,group_stage,regular,110.0,112.0,10.0,28.0,4.0,8.0,4.0,15.0,0.0,4.0,2.0,1.0,18.0,18.0
4,2008-04-20,5,Mumbai,Mumbai Indians,Royal Challengers Bangalore,Royal Challengers Bangalore,MV Boucher,Mumbai Indians,bat,J Srinath,SN Bandekar,AV Jayaprakash,SJ Davis,DJ Harper,5.0,2008,WankhedeStadium,NaN,group_stage,regular,165.0,166.0,11.0,5.0,6.0,0.0,3.0,5.0,2.0,0.0,0.0,0.0,19.0,19.0


In [18]:
cols = [
    "date",
    "match_number",
    "city",
    "team_1",
    "team_2",
    "team_1_runs.total",
    "team_2_runs.total",
    "outcome_winner",
    "player_of_match",
    "team_1_over",
    "team_2_over",
    "toss_winner",
    "toss_decision",
    "team_1_runs.extras",
    "team_2_runs.extras",
    "team_1_extras.legbyes",
    "team_2_extras.legbyes",
    "team_1_extras.wides",
    "team_2_extras.wides",
    "team_1_extras.byes",
    "team_2_extras.byes",
    "team_1_extras.noballs",
    "team_2_extras.noballs",
    "officials_match_referees",
    "officials_reserve_umpires",
    "officials_tv_umpires",
    "officials_umpires_1",
    "officials_umpires_2",
    "outcome_by_wickets",
    "season",
    "venue",
    "outcome_by_runs",
    "event_stage",
    "outcome_method",
]
df_merged = df_merged.reindex(columns=cols)

In [19]:
pd.set_option('display.max_columns', 40)

df_merged.head()

,date,match_number,city,team_1,team_2,team_1_runs.total,team_2_runs.total,outcome_winner,player_of_match,team_1_over,team_2_over,toss_winner,toss_decision,team_1_runs.extras,team_2_runs.extras,team_1_extras.legbyes,team_2_extras.legbyes,team_1_extras.wides,team_2_extras.wides,team_1_extras.byes,team_2_extras.byes,team_1_extras.noballs,team_2_extras.noballs,officials_match_referees,officials_reserve_umpires,officials_tv_umpires,officials_umpires_1,officials_umpires_2,outcome_by_wickets,season,venue,outcome_by_runs,event_stage,outcome_method
0,2008-04-18,1,Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,222.0,82.0,Kolkata Knight Riders,BB McCullum,19.0,15.0,Royal Challengers Bangalore,field,17.0,19.0,4.0,8.0,9.0,11.0,4.0,0.0,0.0,0.0,J Srinath,VN Kulkarni,AM Saheba,Asad Rauf,RE Koertzen,NaN,2008,MChinnaswamyStadium,140.0,group_stage,regular
1,2008-04-19,2,Chandigarh,Chennai Super Kings,Kings XI Punjab,240.0,207.0,Chennai Super Kings,MEK Hussey,19.0,19.0,Chennai Super Kings,bat,6.0,11.0,2.0,4.0,3.0,5.0,0.0,2.0,1.0,0.0,S Venkataraghavan,MSS Ranawat,RB Tiffin,MR Benson,SL Shastri,NaN,2008,PunjabCricketAssociationStadium,33.0,group_stage,regular
2,2008-04-19,3,Delhi,Rajasthan Royals,Delhi Daredevils,129.0,132.0,Delhi Daredevils,MF Maharoof,19.0,15.0,Rajasthan Royals,bat,7.0,10.0,3.0,0.0,3.0,10.0,1.0,0.0,0.0,0.0,GR Viswanath,NaN,IL Howell,Aleem Dar,GA Pratapkumar,9.0,2008,FerozShahKotla,NaN,group_stage,regular
3,2008-04-20,4,Kolkata,Deccan Chargers,Kolkata Knight Riders,110.0,112.0,Kolkata Knight Riders,DJ Hussey,18.0,18.0,Deccan Chargers,bat,10.0,28.0,4.0,8.0,4.0,15.0,0.0,4.0,2.0,1.0,FM Engineer,F Gomes,Asad Rauf,BF Bowden,K Hariharan,5.0,2008,EdenGardens,NaN,group_stage,regular
4,2008-04-20,5,Mumbai,Mumbai Indians,Royal Challengers Bangalore,165.0,166.0,Royal Challengers Bangalore,MV Boucher,19.0,19.0,Mumbai Indians,bat,11.0,5.0,6.0,0.0,3.0,5.0,2.0,0.0,0.0,0.0,J Srinath,SN Bandekar,AV Jayaprakash,SJ Davis,DJ Harper,5.0,2008,WankhedeStadium,NaN,group_stage,regular


In [20]:
df_merged.rename(columns={"outcome_winner": "match_winner"}, inplace=True)

In [21]:
df_merged["match_extras"] = df_merged["team_1_runs.extras"] + df_merged["team_2_runs.extras"]

In [22]:
df_merged["match_legbyes"] = df_merged["team_1_extras.legbyes"] + df_merged["team_2_extras.legbyes"]

In [23]:
df_merged["match_wides"] = df_merged["team_1_extras.wides"] + df_merged["team_2_extras.wides"]

In [24]:
df_merged["match_byes"] = df_merged["team_1_extras.byes"] + df_merged["team_2_extras.byes"]

In [25]:
df_merged["match_byes"] = df_merged["team_1_extras.byes"] + df_merged["team_2_extras.byes"]

In [26]:
df_merged["match_noballs"] = df_merged["team_1_extras.noballs"] + df_merged["team_2_extras.noballs"]

In [27]:
df_merged.head()

,date,match_number,city,team_1,team_2,team_1_runs.total,team_2_runs.total,match_winner,player_of_match,team_1_over,team_2_over,toss_winner,toss_decision,team_1_runs.extras,team_2_runs.extras,team_1_extras.legbyes,team_2_extras.legbyes,team_1_extras.wides,team_2_extras.wides,team_1_extras.byes,team_2_extras.byes,team_1_extras.noballs,team_2_extras.noballs,officials_match_referees,officials_reserve_umpires,officials_tv_umpires,officials_umpires_1,officials_umpires_2,outcome_by_wickets,season,venue,outcome_by_runs,event_stage,outcome_method,match_extras,match_legbyes,match_wides,match_byes,match_noballs
0,2008-04-18,1,Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,222.0,82.0,Kolkata Knight Riders,BB McCullum,19.0,15.0,Royal Challengers Bangalore,field,17.0,19.0,4.0,8.0,9.0,11.0,4.0,0.0,0.0,0.0,J Srinath,VN Kulkarni,AM Saheba,Asad Rauf,RE Koertzen,NaN,2008,MChinnaswamyStadium,140.0,group_stage,regular,36.0,12.0,20.0,4.0,0.0
1,2008-04-19,2,Chandigarh,Chennai Super Kings,Kings XI Punjab,240.0,207.0,Chennai Super Kings,MEK Hussey,19.0,19.0,Chennai Super Kings,bat,6.0,11.0,2.0,4.0,3.0,5.0,0.0,2.0,1.0,0.0,S Venkataraghavan,MSS Ranawat,RB Tiffin,MR Benson,SL Shastri,NaN,2008,PunjabCricketAssociationStadium,33.0,group_stage,regular,17.0,6.0,8.0,2.0,1.0
2,2008-04-19,3,Delhi,Rajasthan Royals,Delhi Daredevils,129.0,132.0,Delhi Daredevils,MF Maharoof,19.0,15.0,Rajasthan Royals,bat,7.0,10.0,3.0,0.0,3.0,10.0,1.0,0.0,0.0,0.0,GR Viswanath,NaN,IL Howell,Aleem Dar,GA Pratapkumar,9.0,2008,FerozShahKotla,NaN,group_stage,regular,17.0,3.0,13.0,1.0,0.0
3,2008-04-20,4,Kolkata,Deccan Chargers,Kolkata Knight Riders,110.0,112.0,Kolkata Knight Riders,DJ Hussey,18.0,18.0,Deccan Chargers,bat,10.0,28.0,4.0,8.0,4.0,15.0,0.0,4.0,2.0,1.0,FM Engineer,F Gomes,Asad Rauf,BF Bowden,K Hariharan,5.0,2008,EdenGardens,NaN,group_stage,regular,38.0,12.0,19.0,4.0,3.0
4,2008-04-20,5,Mumbai,Mumbai Indians,Royal Challengers Bangalore,165.0,166.0,Royal Challengers Bangalore,MV Boucher,19.0,19.0,Mumbai Indians,bat,11.0,5.0,6.0,0.0,3.0,5.0,2.0,0.0,0.0,0.0,J Srinath,SN Bandekar,AV Jayaprakash,SJ Davis,DJ Harper,5.0,2008,WankhedeStadium,NaN,group_stage,regular,16.0,6.0,8.0,2.0,0.0


In [28]:
df.head(10)

,date,match_number,innings,over,batter,bowler,non_striker,runs.batter,runs.extras,runs.total,extras.legbyes,extras.wides,extras.byes,extras.noballs,wicket.kind,wicket.player_out,wicket.fielders,wickets
0,2008-04-18,1,1,0,SC Ganguly,P Kumar,BB McCullum,0,1,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2008-04-18,1,1,0,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2008-04-18,1,1,0,BB McCullum,P Kumar,SC Ganguly,0,1,1,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2008-04-18,1,1,0,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2008-04-18,1,1,0,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2008-04-18,1,1,0,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2008-04-18,1,1,0,BB McCullum,P Kumar,SC Ganguly,0,1,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2008-04-18,1,1,1,BB McCullum,Z Khan,SC Ganguly,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2008-04-18,1,1,1,BB McCullum,Z Khan,SC Ganguly,4,0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2008-04-18,1,1,1,BB McCullum,Z Khan,SC Ganguly,4,0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
runs_groupby= df.groupby(['date', 'match_number', 'runs.batter'])['runs.batter'].agg(["count"])

In [30]:
runs_groupby

count
date       match_number runs.batter       
2008-04-18 1            0              110
                        1               66
                        2               14
                        4               18
                        6               17
...                                    ...
2024-05-26 Final        0               90
                        1               57
                        2               10
                        4               18
                        6                9

[6068 rows x 1 columns]

In [31]:
runs_df = runs_groupby.reset_index()

In [32]:
runs_df = runs_df[(runs_df['runs.batter'] == 4) | (runs_df['runs.batter'] == 6) | (runs_df['runs.batter'] == 0) ]

In [33]:
runs_df

,date,match_number,runs.batter,count
0,2008-04-18,1,0,110
3,2008-04-18,1,4,18
4,2008-04-18,1,6,17
5,2008-04-19,2,0,72
8,2008-04-19,2,4,38
...,...,...,...,...
6061,2024-05-24,Qualifier 2,4,24
6062,2024-05-24,Qualifier 2,6,14
6063,2024-05-26,Final,0,90
6066,2024-05-26,Final,4,18


In [34]:
runs_pivot = runs_df.pivot(index=["date", "match_number"], columns="runs.batter", values=["count"])

In [35]:
runs_pivot

count        
runs.batter                 0   4   6
date       match_number              
2008-04-18 1              110  18  17
2008-04-19 2               72  38  25
           3               97  32   4
2008-04-20 4              147  11  10
           5               94  33  11
...                       ...  ..  ..
2024-05-19 69              82  31  26
2024-05-21 Qualifier 1     80  29  16
2024-05-22 Eliminator      81  33  13
2024-05-24 Qualifier 2    107  24  14
2024-05-26 Final           90  18   9

[1095 rows x 3 columns]

In [36]:
runs_pivot.reset_index(inplace=True)

In [37]:
runs_pivot.columns.values

array([('date', ''), ('match_number', ''), ('count', 0), ('count', 4),
       ('count', 6)], dtype=object)

In [38]:
runs_pivot.columns = [col[0] + "_" + str(col[1]) for col in runs_pivot.columns.values]

In [39]:
runs_pivot.columns

Index(['date_', 'match_number_', 'count_0', 'count_4', 'count_6'], dtype='object')

In [40]:
runs_pivot = runs_pivot.rename(columns={"date_": "date", "match_number_": "match_number", "count_0": "match_dotballs" , "count_4": "match_4's" , "count_6": "match_6's"} )


In [41]:
runs_pivot

,date,match_number,match_dotballs,match_4's,match_6's
0,2008-04-18,1,110,18,17
1,2008-04-19,2,72,38,25
2,2008-04-19,3,97,32,4
3,2008-04-20,4,147,11,10
4,2008-04-20,5,94,33,11
...,...,...,...,...,...
1090,2024-05-19,69,82,31,26
1091,2024-05-21,Qualifier 1,80,29,16
1092,2024-05-22,Eliminator,81,33,13
1093,2024-05-24,Qualifier 2,107,24,14


In [44]:
df_merged = pd.merge(df_merged, runs_pivot, how="outer", on=["date", "match_number"])

In [45]:
df_merged

,date,match_number,city,team_1,team_2,team_1_runs.total,team_2_runs.total,match_winner,player_of_match,team_1_over,team_2_over,toss_winner,toss_decision,team_1_runs.extras,team_2_runs.extras,team_1_extras.legbyes,team_2_extras.legbyes,team_1_extras.wides,team_2_extras.wides,team_1_extras.byes,...,officials_tv_umpires,officials_umpires_1,officials_umpires_2,outcome_by_wickets,season,venue,outcome_by_runs,event_stage,outcome_method,match_extras,match_legbyes,match_wides,match_byes,match_noballs,match_dotballs_x,match_4's_x,match_6's_x,match_dotballs_y,match_4's_y,match_6's_y
0,2008-04-18,1,Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,222.0,82.0,Kolkata Knight Riders,BB McCullum,19.0,15.0,Royal Challengers Bangalore,field,17.0,19.0,4.0,8.0,9.0,11.0,4.0,...,AM Saheba,Asad Rauf,RE Koertzen,NaN,2008,MChinnaswamyStadium,140.0,group_stage,regular,36.0,12.0,20.0,4.0,0.0,110,18,17,110,18,17
1,2008-04-19,2,Chandigarh,Chennai Super Kings,Kings XI Punjab,240.0,207.0,Chennai Super Kings,MEK Hussey,19.0,19.0,Chennai Super Kings,bat,6.0,11.0,2.0,4.0,3.0,5.0,0.0,...,RB Tiffin,MR Benson,SL Shastri,NaN,2008,PunjabCricketAssociationStadium,33.0,group_stage,regular,17.0,6.0,8.0,2.0,1.0,72,38,25,72,38,25
2,2008-04-19,3,Delhi,Rajasthan Royals,Delhi Daredevils,129.0,132.0,Delhi Daredevils,MF Maharoof,19.0,15.0,Rajasthan Royals,bat,7.0,10.0,3.0,0.0,3.0,10.0,1.0,...,IL Howell,Aleem Dar,GA Pratapkumar,9.0,2008,FerozShahKotla,NaN,group_stage,regular,17.0,3.0,13.0,1.0,0.0,97,32,4,97,32,4
3,2008-04-20,4,Kolkata,Deccan Chargers,Kolkata Knight Riders,110.0,112.0,Kolkata Knight Riders,DJ Hussey,18.0,18.0,Deccan Chargers,bat,10.0,28.0,4.0,8.0,4.0,15.0,0.0,...,Asad Rauf,BF Bowden,K Hariharan,5.0,2008,EdenGardens,NaN,group_stage,regular,38.0,12.0,19.0,4.0,3.0,147,11,10,147,11,10
4,2008-04-20,5,Mumbai,Mumbai Indians,Royal Challengers Bangalore,165.0,166.0,Royal Challengers Bangalore,MV Boucher,19.0,19.0,Mumbai Indians,bat,11.0,5.0,6.0,0.0,3.0,5.0,2.0,...,AV Jayaprakash,SJ Davis,DJ Harper,5.0,2008,WankhedeStadium,NaN,group_stage,regular,16.0,6.0,8.0,2.0,0.0,94,33,11,94,33,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,2024-05-19,69,Hyderabad,Punjab Kings,Sunrisers Hyderabad,214.0,215.0,Sunrisers Hyderabad,Abhishek Sharma,19.0,19.0,Punjab Kings,bat,10.0,17.0,5.0,6.0,4.0,10.0,0.0,...,HAS Khalid,Nitin Menon,VK Sharma,4.0,2024,RajivGandhiInternationalStadium,NaN,group_stage,regular,27.0,11.0,14.0,0.0,2.0,82,31,26,82,31,26
1091,2024-05-21,Qualifier 1,Ahmedabad,Sunrisers Hyderabad,Kolkata Knight Riders,159.0,164.0,Kolkata Knight Riders,MA Starc,19.0,13.0,Sunrisers Hyderabad,bat,7.0,11.0,0.0,9.0,5.0,2.0,0.0,...,KN Ananthapadmanabhan,AK Chaudhary,R Pandit,8.0,2024,NarendraModiStadium,NaN,Qualifier 1,regular,18.0,9.0,7.0,0.0,2.0,80,29,16,80,29,16
1092,2024-05-22,Eliminator,Ahmedabad,Royal Challengers Bengaluru,Rajasthan Royals,172.0,174.0,Rajasthan Royals,R Ashwin,19.0,18.0,Rajasthan Royals,field,4.0,6.0,2.0,1.0,2.0,5.0,0.0,...,AK Chaudhary,KN Ananthapadmanabhan,MV Saidharshan Kumar,4.0,2024,NarendraModiStadium,NaN,Eliminator,regular,10.0,3.0,7.0,0.0,0.0,81,33,13,81,33,13
1093,2024-05-24,Qualifier 2,Chennai,Sunrisers Hyderabad,Rajasthan Royals,175.0,139.0,Sunrisers Hyderabad,Shahbaz Ahmed,19.0,19.0,Rajasthan Royals,field,8.0,5.0,1.0,1.0,7.0,3.0,0.0,...,MA Gough,Nitin Menon,VK Sharma,NaN,2024,MAChidambaramStadium,36.0,Qualifier 2,regular,13.0,2.0,10.0,0.0,1.0,107,24,14,107,24,14


In [47]:
df_merged.to_csv("output/ipl_match_level_stats.csv", index=False)